## Environment

In [ ]:
# run in cellpose_2 conda environment (anaconda terminal) in the alien computer
import os
import time
import numpy as np
import tifffile
import cupy
import cupyx.scipy.ndimage

import concurrent.futures 

from skimage.morphology import white_tophat, black_tophat, disk
from skimage import io, measure, util, morphology

from scipy import ndimage
from scipy.sparse import csr matrix

from cellpose import models, io
from tqdm import tqdm

## Read image

In [ ]:
# read image
file_name = ""
img = io.imread(file_name)

print(img.shape)
print(img.dtype)

## Get nuclear masks
 - Nuclear mask --> for nuclear signal intensities
 - Dilated nuclear mask --> for cytoplasmic signal intensities

In [ ]:
# - - - - - Nuclear mask

## segmentation channel
seg_img = img[3, :, :]

## cellpose parameters
diameter = 30
flow_threshold = 0
mask_threshold = -6
do_3D = True
stitch_threshold = 0
gpu = True

## run cellpose in 3D mode
model = models.Cellpose(gpu = gpu, model_type = 'cyto')
masks, flows, styles, diams = model.eval(
    seg_img, 
    channels = [0,0], 
    diameter = diameter, 
    do_3D = do_3D, 
    flow_threshold = flow_threshold, 
    cellprob_threshold = mask_threshold
    )

# - - - - - Dilated nuclear mask 

## helper functions
def dilate_mask_cupy(segmentation_array, label_id, dilation_iterations):
    current_label_id = cupy.where(cupy.array(segmentation_array) == label_id, 1, 0)
    dilated = cupyx.scipy.ndimage.binary_dilation(cupy.array(current_label_id), iterations = dilation_iterations)
    relabeled_dilated = cupy.where(dilated == 1, label_id, 0)
    return(np.array(relabeled_dilated))

def dilate_labels_cupy(segmentation, dilation_iterations):
    list_of_dilated_masks = list()
    regions = measure.regionprops(segmentation)
    for i in range(len(regions)):
        label_id = regions[i].label
        list_of_dilated_masks.append(dilate_mask_cupy(segmentation, label_id, dilation_iterations))
    final_array_labelled = sum(list_of_dilated_masks)
    return(final_array_labelled)

## get dilated mask
mask_dilated = dilate_labels_cupy(mask, dilation_iterations = 2)

## get doughnut mask
mask_doughnut = mask_dilated - mask

# - - - - - Save mask images

## save mask image
mask_output_path = f"{os.path.basename(file_name)[:-4]}_dia-{diameter}_3D-{do_3D}_stitch-{stitch_threshold}.tif"
io.imsave(mask_output_path, masks)

## Calculate fluorescence intensities

In [ ]:
# - - - - - 
